# Gradio Code

* Add overview of what notebooks does and instructions for use

* I would suggest breaking the single code cell into chunks and adding some explanations above each (you can then save the notebook as a .py file if you want a script that will run).

In [5]:
import os
import json
import requests
import gradio as gr
from dotenv import load_dotenv
from datetime import datetime
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.tools import BaseTool


_ = load_dotenv()


# History Setup
HISTORY_DIR = "tmp/shared_betting_history"
os.makedirs(HISTORY_DIR, exist_ok=True)

try:
    os.chmod(HISTORY_DIR, 0o777)  # Allow global access (read/write)
except PermissionError:
    print(f"⚠️ Could not change permissions for {HISTORY_DIR}. You may need to set them manually.")


def get_user_history_file(user_id):
    return os.path.join(HISTORY_DIR, f"{user_id}.json")

def load_user_history(user_id):
    path = get_user_history_file(user_id)
    if os.path.exists(path):
        with open(path, "r") as f:
            data = json.load(f)
        return data.get("chat_history", []), data.get("agent_scratchpad", [])
    return [], []

def save_user_history(user_id, chat_history, agent_scratchpad):
    path = get_user_history_file(user_id)
    with open(path, "w") as f:
        json.dump({
            "chat_history": [m.dict() for m in chat_history],
            "agent_scratchpad": agent_scratchpad,
        }, f, indent=2)
    os.chmod(path, 0o666)


class NBAOddsTool(BaseTool):
    name: str = "nba_odds_api"
    description: str = (
        "Use this to get current NBA game odds from The Odds API. Input is ignored."
    )

    def _run(self, _: str = "") -> str:
        api_key = os.getenv("ODDS_API_KEY")
        url = (
            "https://api.the-odds-api.com/v4/sports/basketball_nba/odds"
            f"?apiKey={api_key}&regions=us&markets=h2h,spreads,totals"
        )
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        return resp.text

    async def _arun(self, _: str):
        raise NotImplementedError

nba_odds_tool = NBAOddsTool()
tavily_tool = TavilySearchResults(k=5, time_period="day")
TOOLS = [nba_odds_tool, tavily_tool]

today = datetime.today().strftime("%B %d, %Y")

# read system prompt from file
SYSTEM_PROMPT = open('system_prompt.txt').read().replace('{today}', today)

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    MessagesPlaceholder("chat_history"),
    MessagesPlaceholder("agent_scratchpad"),
    ("human", "{input}")
])

#LLM Setup
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)
agent = create_openai_functions_agent(llm=llm, tools=TOOLS, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=TOOLS, verbose=True, handle_parsing_errors=True)

session_cache = {}

# Welcome message for returning user
def welcome_user(user_id):
    chat, scratchpad = load_user_history(user_id)
    if not chat:
        return [], [], None

    last_msg = next((m["content"] for m in reversed(chat) if m["type"] == "ai"), None)
    if not last_msg:
        return [], [], None

    welcome_prompt = (
        f"The user has returned. Their last assistant message was:\n\"{last_msg}\"\n"
        f"Generate a warm welcome that references this bet and invites them to continue."
    )
    response = llm.invoke(welcome_prompt)
    return [HumanMessage(**m) if m["type"] == "human" else AIMessage(**m) for m in chat], scratchpad, response.content

# Chat stuff
def chat_fn(message, user_id):
    if not user_id:
        return "Please enter your username to continue.", []

    if user_id not in session_cache:
        chat_history, scratchpad, welcome = welcome_user(user_id)
        display = [(None, welcome)] if welcome else []
        session_cache[user_id] = {
            "chat_history": chat_history,
            "agent_scratchpad": scratchpad,
            "display": display
        }

    session = session_cache[user_id]
    session["chat_history"].append(HumanMessage(content=message))

    response = agent_executor.invoke({
        "input": message,
        "chat_history": session["chat_history"],
        "agent_scratchpad": session["agent_scratchpad"]
    })

    session["chat_history"].append(AIMessage(content=response["output"]))
    session["display"].append((message, response["output"]))
    save_user_history(user_id, session["chat_history"], session["agent_scratchpad"])

    return "", session["display"]

# Interface stuff
with gr.Blocks() as app:
    gr.Markdown("## 🏀 NBA Betting Assistant")

    user_input = gr.Textbox(label="Username", placeholder="e.g. matt24")
    chatbot = gr.Chatbot()
    message_input = gr.Textbox(label="Message", placeholder="Ask about tonight’s games…")
    send_btn = gr.Button("Send")



    def initialize_user(uid):
        if uid and uid not in session_cache:
            _, _, welcome = welcome_user(uid)
            if welcome:
                session_cache[uid] = {
                    "chat_history": [],
                    "agent_scratchpad": [],
                    "display": [(None, welcome)]
                }
                return [(None, welcome)]
            else:
                session_cache[uid] = {"chat_history": [], "agent_scratchpad": [], "display": []}
        return session_cache.get(uid, {}).get("display", [])

    user_input.change(initialize_user, inputs=user_input, outputs=chatbot)
    send_btn.click(chat_fn, inputs=[message_input, user_input], outputs=[message_input, chatbot])




⚠️ Could not change permissions for tmp/shared_betting_history. You may need to set them manually.


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [6]:
app.launch(share=True)

NameError: name 'app' is not defined

In [11]:
app.close()